In [74]:
import requests
import pandas as pd
import base_tools as tools
import numpy as np
from collections import defaultdict, OrderedDict
import operator
import json
from os.path import join as pjoin
import time
import os

https://api.opendota.com/api/matches/

In [75]:


filename = "archieved_match.json"
path_to_file = "D:\\Pro\\Py-DOta\\Dumped_Json\\archieved_match.json"
with open(path_to_file,'r') as f:
    data = json.load(f)
archieved_match = data

In [76]:
def post(df):
    df = df.groupby(df.columns, axis=1).sum()
    df['AVG_FS'] = df.apply(lambda row:row[0]/row[2],axis=1)
    df = df.sort_values(by=['AVG_FS'],ascending=False)
    df.columns = ['FS','Region','Matches','AVG_FS']
    return df


In [77]:
def get_fs_for_one_match(match_id):
    player_table = {}
    filename = str(match_id) + ".json"
    path_to_file = pjoin("D:\\Pro","Py-DOta","Dumped_Json",filename)
    if match_id in archieved_match:

        with open(path_to_file,'r') as f:
            details = json.load(f)
            f.close()
    else:
        target = "https://api.opendota.com/api/matches/" + str(match_id) 
        r = requests.get(url=target)
        time.sleep(2)
        details = r.json()
        with open(path_to_file,'w+') as f:
            json.dump(details,f)
            f.close()
        archieved_match.append(match_id)
        path_to_file = "D:\\Pro\\Py-DOta\\Dumped_Json\\archieved_match.json"
        with open(path_to_file,'w') as f:
            json.dump(archieved_match,f)
            f.close()
        
    try:

        fb_time = details['first_blood_time']
        radiant_kills = details['radiant_score']
        dire_kills = details['dire_score']
        #print(details.keys())
        try:
            region = details['region']
        except:
            region = 12

        
        for player in details['players']:
            #player_stats = []
            kills = player['kills']
            deaths = player['deaths']
            assists = player['assists']
            try:
                first_kill_time = player['kills_log'][0]['time']
            except:
                first_kill_time = -100
            if first_kill_time == fb_time:
                fb = 1
            else:
                fb = 0
            creep_scores = player['denies'] + player['last_hits']
            wards = player['obs_placed'] + player['sen_placed']
            camp = player['camps_stacked']
            stuns = player['stuns']
            gpm = player['gold_per_min']
            runes = player['rune_pickups']
            tower_kill = player['tower_kills']
            if player['isRadiant']:
                participant = (kills+assists)/radiant_kills
            else:
                participant = (kills+assists)/dire_kills
            roshan_kills = player['roshan_kills']
            player_fs = kills *0.3 - 0.3* deaths + 4.0* fb+ 0.003 * creep_scores+ 0.5*wards+0.5*camp+0.05*stuns+0.002*gpm+ 0.25*runes+tower_kill+3*participant+roshan_kills
            player_table[player['name']] = [player_fs,player['lane_role'],player['lh_t'][9],region,player['account_id']]
    except Exception as e:
        player_table[match_id] = str(e)
    return player_table

        

In [78]:
def get_fs_for_one_league(match_list,qualifier = False):
    all_table = {}
    mid_table = {}
    support_table = {}
    core_table = {}
    Error_ID = {}
    for match_id in match_list:

        player_table = get_fs_for_one_match(match_id)
        if len(player_table) == 1:
            k = list(player_table.keys())
            v = list(player_table.values())
            Error_ID[k[0]] = v[0]
            continue

        for k, v in player_table.items():
            if k in all_table:
                all_table[k][0] = all_table[k][0] + v[0] # total FS
                all_table[k][1].append(v[1]) # positions played
                all_table[k][2] = (all_table[k][2] + v[2])/2 # average lh@10mins
                all_table[k][4] = all_table[k][4]+ 1 # total matches played
            else:
                all_table[k] = [v[0],[v[1]],v[2],v[3],1,v[4]]
    for player,stats in all_table.items():
        role_list = stats[1]
        role_list.sort(reverse = True)
        d = defaultdict(int)
        for i in role_list:
            d[i] += 1
        (role,role_times) = max(d.items(),key=lambda x: x[1])
        if player == None:
            player = stats[5]
        if player == 'JAbz':
            print(role_list)
        lh_10_min = stats[2]
        if role == 2 and lh_10_min > 20:
            if qualifier:
                mid_table[player] = [stats[0],stats[3],stats[4]]
            else:
                mid_table[player] = [stats[0],stats[3],stats[4]]

                
        else:
            # if a plyaer have an average lh at 10 mins > 20, the player is regarded as a core.
            try:
                if lh_10_min > 20:
                    if qualifier:
                        core_table[player] = [stats[0],stats[3],stats[4]]
                    else:
                        core_table[player] = [stats[0],stats[3],stats[4]]
                else:
                    if qualifier:
                        support_table[player] = [stats[0],stats[3],stats[4]]
                    else:
                        support_table[player] = [stats[0],stats[3],stats[4]]
            except:
                print(lh_10_min)
    return all_table, mid_table,support_table, core_table,Error_ID

                

In [7]:
table, qualifier,main_event = tools.differed_id("Paris Major")


In [79]:
leagues = ["PIT Minor","DreamLeague S10","DreamLeague S11","SL-i Minor","Kuala Lumpur Major","Chongqing Major","Paris Major","Bucharest Minor"]

In [45]:
leagues = ["PIT Minor"]

In [9]:
all_table, mid_table, support_table, core_table,Error_ID = get_fs_for_one_league(qualifier,False)
Error_ID

{'4598714044': "unsupported operand type(s) for +: 'NoneType' and 'NoneType'",
 '4583318221': 'division by zero',
 '4581422908': "unsupported operand type(s) for +: 'NoneType' and 'NoneType'",
 '4581417620': "unsupported operand type(s) for +: 'NoneType' and 'NoneType'"}

In [80]:
mid_df_list = []
sup_df_list = []
core_df_list = []
mid_df_list2 = []
sup_df_list2 = []
core_df_list2 = []
cols = ['FS','Region','Matches','ID']
for league in leagues:
    table, qualifier,main_event = tools.differed_id(league)
    all_table, mid_table, support_table, core_table,Error_ID = get_fs_for_one_league(qualifier,False)
    df = pd.DataFrame(mid_table)
    df = df.T
    mid_df_list.append(df)
    df2 = pd.DataFrame(core_table)
    df2 = df2.T    
    core_df_list.append(df2)
    df3 = pd.DataFrame(support_table)
    df3 = df3.T  
    sup_df_list.append(df3)
    all_table, mid_table, support_table, core_table,Error_ID = get_fs_for_one_league(main_event,False)
    df = pd.DataFrame(mid_table)
    df = df.T
    mid_df_list2.append(df)
    df2 = pd.DataFrame(core_table)
    df2 = df2.T    
    core_df_list2.append(df2)
    df3 = pd.DataFrame(support_table)
    df3 = df3.T  
    sup_df_list2.append(df3)
mid_df = post(pd.concat(mid_df_list,axis = 1,sort=True))
mid_df.to_excel("mid_df_qu.xlsx")
core_df = post(pd.concat(core_df_list,axis = 1,sort=True))
core_df.to_excel("core_df_qu.xlsx")
sup_df = post(pd.concat(sup_df_list,axis = 1,sort=True))
sup_df.to_excel("sup_df_qu.xlsx")
mid_df2 = post(pd.concat(mid_df_list2,axis = 1,sort=True))
mid_df2.to_excel("mid_df_main.xlsx")
core_df2 = post(pd.concat(core_df_list2,axis = 1,sort=True))
core_df2.to_excel("core_df_main.xlsx")
sup_df2 = post(pd.concat(sup_df_list2,axis = 1,sort=True))
sup_df2.to_excel("sup_df_main.xlsx")


[4, 3, 3, 2, 2, 2, 1]
v
[3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1]
[3, 3, 3, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1]
[3, 3, 2, 2, 2, 1, 1, 1, 1]
[3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1]
[3, 3, 3, 3, 2, 2, 2, 1, 1]


In [83]:
mid_df2.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>FS</th>\n      <th>Region</th>\n      <th>Matches</th>\n      <th>AVG_FS</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Daxak</th>\n      <td>227.319654</td>\n      <td>3.0</td>\n      <td>15.0</td>\n      <td>15.154644</td>\n    </tr>\n    <tr>\n      <th>MagicaL</th>\n      <td>393.606255</td>\n      <td>16.0</td>\n      <td>29.0</td>\n      <td>13.572629</td>\n    </tr>\n    <tr>\n      <th>Limmp</th>\n      <td>135.417211</td>\n      <td>8.0</td>\n      <td>10.0</td>\n      <td>13.541721</td>\n    </tr>\n    <tr>\n      <th>Chessie</th>\n      <td>143.815581</td>\n      <td>8.0</td>\n      <td>11.0</td>\n      <td>13.074144</td>\n    </tr>\n    <tr>\n      <th>Abed</th>\n      <td>560.496207</td>\n      <td>33.0</td>\n      <td>44.0</td>\n      <td>12.738550</td>\n    </tr>\n    <tr>\n      <th>201594424</th>\n      <td>210.264448</td>\n      <td>8.0</td>\n

In [12]:
df2 = pd.DataFrame(mid_table)
df2 = df2.T

df2

,0,1,2
ASD,147.508602,3.0,12.0
Afoninje,138.652341,3.0,15.0
Daxak,227.319654,3.0,15.0
一,158.755216,3.0,16.0
Fata,139.370094,3.0,14.0
Topson,93.408445,3.0,11.0
Ryoya,95.368134,3.0,11.0
Panthera,23.069155,3.0,6.0
Faker-,27.033260,3.0,6.0
_Mikoto_,91.285292,3.0,9.0


In [13]:
df3 = pd.concat([df,df2],axis = 1,sort=True)
df3.to_excel("df3.xlsx")

In [69]:

mid_df3 = mid_df2.groupby(mid_df2.columns, axis=1).sum()
mid_df3['AVG_FS'] = mid_df3.apply(lambda row:row[0]/row[2],axis=1)
mid_df3 = mid_df3.sort_values(by=['AVG_FS'],ascending=False)

In [ ]:
mid_df3.


In [72]:
df22 = post(mid_df2)
df22

,FS,Region,Matches,AVG_FS
Daxak,227.319654,3.0,15.0,15.154644
MagicaL,393.606255,16.0,29.0,13.572629
Limmp,135.417211,8.0,10.0,13.541721
Chessie,143.815581,8.0,11.0,13.074144
Abed,560.496207,33.0,44.0,12.738550
201594424,210.264448,8.0,17.0,12.368497
J E I M A R I,84.338869,3.0,7.0,12.048410
小可,273.152632,25.0,23.0,11.876201
fn,141.430101,3.0,12.0,11.785842
430,223.730634,3.0,19.0,11.775297


In [39]:
new_list = []
for df in mid_df_list2:
    if not df.empty:
        new_list.append(df)

mid_df_list2 = new_list
mid_df2 = pd.concat(mid_df_list2,axis = 1,sort=True)

In [56]:
mid_df_list2

[                      0    1     2
 430          223.730634  3.0  19.0
 Fata         246.698020  3.0  20.0
 qojqva       195.615552  3.0  17.0
 Chris Luck   148.091411  3.0  14.0
 CCnC         144.771151  3.0  14.0
 _Mikoto_     157.737942  3.0  13.0
 Afoninje     118.551877  3.0  12.0
 Setsu        124.123239  3.0  11.0,                       0    1     2
 MagicaL      290.990723  8.0  20.0
 k`wonderkid  194.771308  8.0  16.0
 AhJit        214.325249  8.0  16.0
 NaN          210.264448  8.0  17.0
 p4pita       109.752892  8.0  13.0
 Dendi         84.476332  8.0   9.0
 Limmp        135.417211  8.0  10.0
 Chessie      143.815581  8.0  11.0
 iAnnihilate   49.578412  8.0   6.0,                      0    1     2
 Ori         200.955345  8.0  20.0
 No[o]ne-    323.547395  8.0  28.0
 Abed        221.818288  8.0  18.0
 MidOne      168.323364  8.0  16.0
 一           111.333368  8.0  13.0
 Xm          139.984848  8.0  13.0
 SumaiL      152.493374  8.0  13.0
 Bryle        76.703860  8.0   9.0
 

In [41]:
mid_df2

,FS,GG,AA,FS,GG,AA,FS,GG,AA,FS,...,AA,FS,GG,AA,FS,GG,AA,0,1,2
430,223.730634,3.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afoninje,118.551877,3.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,64.887639,5.0,6.0,NaN,NaN,NaN,138.652341,3.0,15.0
AhJit,NaN,NaN,NaN,214.325249,8.0,16.0,NaN,NaN,NaN,NaN,...,NaN,63.810567,5.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
CCnC,144.771151,3.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,66.398831,5.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN
Chris Luck,148.091411,3.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fata,246.698020,3.0,20.0,NaN,NaN,NaN,118.078714,8.0,11.0,NaN,...,NaN,161.526536,5.0,16.0,NaN,NaN,NaN,139.370094,3.0,14.0
Jhocam,155.632131,3.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MagicaL,NaN,NaN,NaN,290.990723,8.0,20.0,102.615532,8.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Setsu,124.123239,3.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,58.570945,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Taiga,177.925748,3.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:

target = "https://api.opendota.com/api/players/201594424"
r = requests.get(url=target)
details = r.json()
details


{'tracked_until': None,
 'solo_competitive_rank': 6439,
 'competitive_rank': None,
 'rank_tier': 80,
 'profile': {'account_id': 201594424,
  'personaname': '.',
  'name': None,
  'plus': True,
  'cheese': 0,
  'steamid': '76561198161860152',
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/6c/6c651b8a543c9e72d9f6851f2dc6549288752320.jpg',
  'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/6c/6c651b8a543c9e72d9f6851f2dc6549288752320_medium.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/6c/6c651b8a543c9e72d9f6851f2dc6549288752320_full.jpg',
  'profileurl': 'https://steamcommunity.com/profiles/76561198161860152/',
  'last_login': None,
  'loccountrycode': None,
  'is_contributor': False},
 'mmr_estimate': {'estimate': 6092},
 'leaderboard_rank': 209}